In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from src.dsets import *
import random

In [21]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [47]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [4]:
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.'
          ]
corpus_embeddings = embedder.encode(corpus)

In [34]:
query = ['Someone is hungry']
query_embedding = embedder.encode(query)

cos_sim = []
for vec in corpus_embeddings:
    cos_sim.append(dot(vec, query_embedding[0])/(norm(vec)*norm(query_embedding[0])))

In [46]:
print("Query:", query[0])
print("\nTop 2 most similar sentences in corpus:")
print("\n")

for idx in np.argsort(cos_sim)[::-1][:2]:
    print(corpus[idx], "(Score: %.4f)" % (cos_sim[idx]))

Query: Someone is hungry

Top 2 most similar sentences in corpus:


A man is eating food. (Score: 0.5234)
A man is eating a piece of bread. (Score: 0.2534)


In [11]:
# lumps that potentially look like nodules
data_candit = pd.read_csv(data_path / 'candidates.csv')
print(len(data_candit))
print(data_candit.seriesuid.nunique())

seriesuid_1 = data_candit.query('`class` == 1').seriesuid
seriesuid_c = data_candit.seriesuid
print(len(seriesuid_1), len(seriesuid_1.unique()))

data_candit.head()

551065
888
1351 590


,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0


In [12]:
# lumps that have been flagged as nodules
data_annot = pd.read_csv(data_path / 'annotations.csv')
print(len(data_annot))
print(data_annot.seriesuid.nunique())
seriesuid_a = data_annot.seriesuid

data_annot.head()

1186
601


,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [31]:
1186/551065*100

0.21521962018999571

In [3]:
%%time
candidateInfo_list = getCandidateInfoList(requireOnDisk_bool=True)

CPU times: user 4.26 s, sys: 409 ms, total: 4.67 s
Wall time: 4.71 s


In [23]:
len(candidateInfo_list)

56938

In [4]:
candidateInfo_list[:5]

[CandidateInfoTuple(isNodule_bool=True, diameter_mm=25.23320204, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803753100045216493273', center_xyz=(63.4740118048, 73.9174523314, -213.736128767)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=25.23320204, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803753100045216493273', center_xyz=(59.1136546856, 82.9816128068, -213.127954744)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=25.23320204, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803753100045216493273', center_xyz=(57.43, 78.25, -221.27)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=21.58311204, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.905371958588660410240398317235', center_xyz=(110.83, 57.69, -122.43)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=21.58311204, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.905371958588660410240398317235', center_xyz=(109.142472723, 49.6356928166, -121.183579092))]

In [6]:
positiveInfo_list = [x for x in candidateInfo_list if x.isNodule_bool]
print(len(positiveInfo_list))
negativeInfo_list = [x for x in candidateInfo_list if not x.isNodule_bool]
print(len(negativeInfo_list))

print(len(positiveInfo_list)/len(negativeInfo_list)*100)

122
56816
0.21472824556462966


In [10]:
diameter_list = [x.diameter_mm for x in positiveInfo_list]

diameter_list[::10]

[25.23320204,
 17.75323185,
 13.84883567,
 11.48732767,
 9.443896197,
 7.98502287,
 7.000523906,
 6.401625027,
 5.965579845,
 5.253525386,
 5.05940652,
 4.343199739,
 0.0]

In [14]:
# nodulelist = findPositiveSamples()

In [15]:
# uid = nodulelist[0].series_uid

In [16]:
# showCandidate(uid)

In [17]:
test_dataset = LunaDataset()

2020-12-12 22:32:02,753 INFO     pid:32 src.dsets:191:__init__ <src.dsets.LunaDataset object at 0x7f2f1c7fd2e0>: 56938 training samples


In [44]:
len(test_dataset)

56938

In [110]:
%%time
k = random.choice(range(len(test_dataset)))
print(k)

14595
CPU times: user 10.6 ms, sys: 8.07 ms, total: 18.6 ms
Wall time: 19.4 ms


In [116]:
%time
obj = test_dataset[k]

CPU times: user 4 µs, sys: 11 µs, total: 15 µs
Wall time: 88.9 µs


In [48]:
print(type(obj))

<class 'tuple'>


In [52]:
obj[3]

tensor([ 52., 333., 356.], dtype=torch.float64)

In [25]:
[x for x in test_dataset.candidateInfo_list if not x.isNodule_bool][-10:]

[CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-137.17, 36.06, -251.99)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-137.87, -13.64, -226.61)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-138.57, -2.44, -264.69)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-140.730526882, 31.1359380816, -237.045267029)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260', center_xyz=(-144.18, 48.66, -297.83)),
 CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.627

In [117]:
N=100

for k in range(400,800):
    sample = test_dataset[k]

In [39]:
%%time

N=100

for k in range(100,N+100):
    sample = test_dataset[k]

CPU times: user 157 ms, sys: 93.1 ms, total: 250 ms
Wall time: 932 ms


199